In [1]:
import docker

client = docker.from_env()
containers = client.containers.list()
for c in containers:
    print(c.image)

<Image: 'milvusdb/milvus-dev:master-20210820-bf2c427'>
<Image: 'quay.io/coreos/etcd:v3.5.0'>
<Image: 'minio/minio:RELEASE.2020-12-03T00-03-10Z'>
<Image: 'registry:2'>


In [5]:
# import
from pymilvus_orm import (
    connections, FieldSchema, CollectionSchema, DataType,
    Collection, list_collections,
)


In [6]:
# create connection and list collection
connections.connect()

print(f"\nList collections...")
col_list = list_collections()
print(f"collections_nums: {len(col_list)}")
# list entities if collections
for name in col_list:
    c = Collection(name = name)
    print(f"{name}: {c.num_entities}")


List collections...
collections_nums: 0


In [7]:
# Create collections and insert data
import random
dim = 128
default_fields = [
    FieldSchema(name="count", dtype=DataType.INT64, is_primary=True),
    FieldSchema(name="random_value", dtype=DataType.DOUBLE), 
    FieldSchema(name="float_vector", dtype=DataType.FLOAT_VECTOR, dim=dim)
]
default_schema = CollectionSchema(fields=default_fields, description="test collection")

print(f"\nCreate collection...")
collection = Collection(name="hello_milvus_v2", schema=default_schema)

print(f"\nList collections...")
print(list_collections())

#  insert data
nb = 3000
vectors = [[random.random() for _ in range(dim)] for _ in range(nb)]
collection.insert(
    [
        [i for i in range(nb)],
        [float(random.randrange(-20, -10)) for _ in range(nb)],
        vectors
    ]
)

print(f"\nGet collection entities...")
print(collection.num_entities)


Create collection...

List collections...
['hello_milvus']

Get collection entities...
3000


In [ ]:
# creat index
default_index = {"index_type": "IVF_FLAT", "params": {"nlist": 128}, "metric_type": "L2"}
col_list = list_collections()
print(f"\nCreate index...")
for name in col_list:
    c = Collection(name = name)

    print(name)
    print(c)
    c.create_index(field_name="float_vector", index_params=default_index)


In [ ]:
# change index
print("change index")
another_index = {"index_type": "IVF_SQ8", "metric_type": "L2", "params": {"nlist": 64}}
col_list = list_collections()
for name in col_list:
    c = Collection(name = name)
    c.create_index(field_name="float_vector", index_params=another_index)

In [ ]:
# drop index
col_list = list_collections()
for name in col_list:
    c = Collection(name=name)
    c.drop_index()

In [ ]:
# load and search
print("search data starts")
col_list = list_collections()
for name in col_list:
    c = Collection(name=name)
    print(f"collection name: {name}")
    c.load()
    topK = 5
    vectors = [[1.0 for _ in range(128)] for _ in range(3000)]
    search_params = {"metric_type": "L2", "params": {"nprobe": 10}}


    import time
    start_time = time.time()
    print(f"\nSearch...")
    # define output_fields of search result
    res = c.search(
        vectors[-2:], "float_vector", search_params, topK,
        "count > 500", output_fields=["count", "random_value"]
    )
    end_time = time.time()
    # show result
    for hits in res:
        for hit in hits:
            # Get value of the random value field for search result
            print(hit, hit.entity.get("random_value"))
        print("###########")
    print("search latency = %.4fs" % (end_time - start_time))
    c.release()
print("search data ends")

In [ ]:
# drop collection
print("Drop Collections")

for name in list_collections():
    c = Collection(name=name)
    c.drop()
# check collection clean up
print(f"\nList collections...")
print(list_collections())